In [1]:
# %load ../startup.py
import os,sys
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

os.environ['PYTHONPATH']=os.environ.get('LIB')
if not os.environ.get('LIB') in sys.path: 
    sys.path.insert(0,os.environ.get('LIB'))

DAT_DIR = os.environ.get('DAT_DIR')    

%load_ext autoreload
%autoreload 2
%pylab inline
%matplotlib inline
%load_ext rpy2.ipython

import matplotlib.text as text
import pandas as pd
import numpy as np
import pylab as pl
import scipy as sp
import sys
import rpy2 
import os 
from matplotlib import gridspec
from scipy.interpolate import splev, splrep
import xlwt
import seaborn as sns
from scipy import stats
import rpy2.robjects.packages as rpackages
import seaborn as sns
from statsmodels import robust


from heprnhci.db.mongo import *


from IPython.core.display import display, HTML
display(HTML("<style>.container {width:80% !important;}</style>")) # increase jupyter screen width to 100%
pd.options.display.max_colwidth = 500
pd.set_option('display.precision',2)

HCI=openMongo(db=os.environ.get('MONGO_HCIDB'),host=os.environ.get('MONGO_HOST'),auth=False)


Populating the interactive namespace from numpy and matplotlib


/opt/miniconda3/envs/tiptox/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/opt/miniconda3/envs/tiptox/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


# Goal 

Check conc-response fits and filter out bad quality hits
Follow instructions on installing ipywidgets (https://ipywidgets.readthedocs.io/en/latest/user_install.html)



In [3]:

# from __future__ import print_function
# from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets

In [4]:
# #CH0 = HCI.ch
# HCI.chemicals.find_one()
# pd.DataFrame(list(HCI.hci_feats.find()))
# #CR0 = Box(HCI.heprn_cr.find_one())

# Design

* Ranked list of hits
* Plot CR for chem, assays, times
* For each CR curve save curve fit:
    * Accept
        * List of accept categories
    * Reject
        * Noisy data
        * Only one point above 3 BMAD
        * Other?
        

## CR Data

In [2]:
from heprnhci.tk.httk import * 


RAT_LIV_LL = pd.read_excel(DAT_DIR+'toxref-v2.0-pods-rat-liver.xlsx')
CH_CAS0=pd.DataFrame(list(HCI.chemicals.find()),
                    columns=['dsstox_sid','chem_name','casrn'])\
        .rename(columns=dict(chem_name='name'))
CAS_TK=getChemInfo()
CAS0 = set(CH_CAS0.casrn).intersection(RAT_LIV_LL.casrn).intersection(CAS_TK)
CH_CAS1 = CH_CAS0[CH_CAS0.casrn.isin(CAS0)]


In [3]:
RES = []

FT_CH = ['MF_dn','DT_up','St_up','NS_dn','Ap_up','CN_dn','St_dn',
         'DT_dn','NS_up','ES_dn','ES_up','LM_up','LM_dn']

Q = dict(timeh={'$in':[24,48,72]},
         name={'$in':list(CH_CAS1.name)})
Q['best_fit.model']={'$ne':'cnst'}
#Q['cr_info.gt_3bmad']=1

for CR in HCI.heprn_cr.find(Q):
    X = pd.DataFrame(CR['cr_data'])
    # Only if there are 3 or more points above bmad
    #if (X.resp.abs()>1).sum()>=1: 
    RES.append(CR['best_fit'])

AC50a = pd.DataFrame(RES)
AC50 = AC50a[(AC50a.ga>=-0.7) & (AC50a.ga<=2) & AC50a.ft_ch.isin(FT_CH)]\
       .merge(CH_CAS0[['name','dsstox_sid']],on='name')\
       .sort_values('ga')
AC50.insert(9,'ac50',10**AC50.ga)
AC50.insert(AC50.shape[1],'hit',[ "%(name)s|%(timeh)dh|%(ft_ch)s|%(ac50)5.2fuM"%i for i in  AC50.to_dict('records')])

X=AC50.pivot_table(index=['name','dsstox_sid','timeh'],
                   columns='ft',
                   values='ac50',
                   aggfunc=np.min)
X.insert(X.shape[1],'lowest',X.min(axis=1))

X=X.sort_values('lowest')
FT = X.columns
HITS=X.reset_index().copy()
HITS.insert(HITS.shape[1],'hits',
            HITS[['name','timeh','lowest']].apply(lambda x: "%30s|%2dh|%5.2fuM" %\
                                          tuple(x),axis=1))

# Run hit evaluation

In [5]:
display(widgets.IntSlider())

IntSlider(value=0)

If you see "IntSlider(value=0)" then your ipywidgets installation is not working!

In [6]:
import seaborn as sns
import re

%matplotlib widget
pl.style.use('default')
from box import Box
    
%config InlineBackend.close_figures=False 

import ipywidgets as widgets
from IPython.display import display,clear_output

from heprnhci.cr.tcpl import *
from heprnhci.viz.cr import *
sns.set(font_scale=0.7,style='ticks')

FT_CH = ['MF_dn','DT_up','St_up','NS_dn','Ap_up','CN_dn','St_dn',
         'DT_dn','NS_up','ES_dn','ES_up','LM_up','LM_dn']

# Save hit quality
HQ = Box()

# Curr chemical

Chem_cur = ''

# Widgets
## Chemical search
O1_rev  = widgets.RadioButtons(description='Reviewed',options=['Yes','No','All'])
T1_chem = widgets.Text(value='',description='Chem name')
S1_hits = widgets.Select(options=AC50.hit,rows=20)

V_search= widgets.VBox([O1_rev,T1_chem,S1_hits])
## r_max
R1 = widgets.FloatSlider(value=20.,min=5.0,max=100.,
                         orientation='vertical',
                         caption=r'r_{max}',
                         layout=widgets.Layout(height='100%')
                        )
B1 = widgets.Button(description='Plot >>',
                    layout=widgets.Layout(width='100px'))
H1 = widgets.HTML(placeholder='Messages',
                  layout=widgets.Layout(width='500px',height='100px'))    
Viz1 = widgets.Output(layout=widgets.Layout(width='500px',height='200px'))
Viz2 = widgets.Output(layout=widgets.Layout(width='2000px',height='1000px'))

## Save 
L1 = widgets.Label(value='Save hit information')
Hit_status=widgets.Label(value='')
Ch_name= widgets.Label(value='')
Asy_name= widgets.Label(value='')
Ch_timeh= widgets.Label(value='')
Ch_ac50= widgets.Text(value='')
Opt_hit = widgets.RadioButtons(description='Hit',options=['Yes','No'])
Opt_qual   = widgets.Dropdown(description='Quality',options=['ok','noisy','lt_3bmad','other'])
Qual_other=widgets.Text(value='')
Qual_user=widgets.Text(value='ishah')
Qual_save=widgets.RadioButtons(description='Save',options=['Del_prev','Append'])

B2 = widgets.Button(description='Save',
                    layout=widgets.Layout(width='100px'))

## Hit quality
V_chem = widgets.VBox([L1,Hit_status,
                       widgets.HBox([widgets.Label(value='Chemical:'),Ch_name]),
                       widgets.HBox([widgets.Label(value='Assay     :'),Asy_name]),
                       widgets.HBox([widgets.Label(value='Time(h) :'),Ch_timeh]),
                       widgets.HBox([widgets.Label(value=r'AC50 $\mu$M:'),Ch_ac50]),
                       Opt_hit,
                       Opt_qual,
                       widgets.HBox([widgets.Label(value='Qual. flags:'),Qual_other]),
                       widgets.HBox([widgets.Label(value='Userid:'),Qual_user]), 
                       widgets.HBox([widgets.Label(value='Record'),Qual_save]), 
                       B2
                      ])

## Extre info
## AED
AED_info = widgets.HTML(value='',style={'font-size':'small'})
## Log
Log_info = widgets.HTML(value='')
## Hits
Hit_info = widgets.HTML(value='')

Info_tab = widgets.Tab(children=[Hit_info,AED_info,Log_info],
                      layout=widgets.Layout(width='50%',height='100%'))
#Info_tab.children=[Hit_info,AED_info,Log_info]
Info_tab.set_title(0,'Hits')
Info_tab.set_title(1,'AEDs')
Info_tab.set_title(2,'Log')

## Layout
L1 = widgets.HBox([V_search,R1,B1,Viz1,V_chem,Info_tab])
L3 = widgets.VBox([L1,Viz2])


# Functions 

def getHits(reviewed='All',chemical=None,HITS=AC50):
    REV=pd.DataFrame(list(HCI.heprn_hitqual.find({},dict(_id=0))))    
    R1 = REV[['name','timeh','ft_ch']].drop_duplicates()
    R2 = R3 = None
    if reviewed=='All': 
        R2 = HITS
    elif reviewed=='Yes':
        R2 = HITS.merge(R1,on=['name','timeh','ft_ch'],how='left')
    elif reviewed=='No':
        X1=pd.DataFrame(list(HCI.heprn_hitqual.find({},dict(_id=0))))[['name','timeh','ft_ch','hit']]\
            .drop_duplicates().rename(columns=dict(hit='hit_call'))
        X2 = AC50.merge(X1,on=['name','timeh','ft_ch'],how='outer')
        R2 = X2[X2.hit_call.isna()].drop('hit_call',axis=1)
        
    if chemical and len(chemical)>2:
        R2 = R2[R2.name.str.match(chemical,case=False)]
        
    return R2

def getHitSummary(chemical,HITS=AC50):
    try:
        Q0=pd.DataFrame(list(HCI.heprn_hitqual.find(dict(name=chemical),dict(_id=0))))\
             .sort_values('ac50')
    except:
        return ''
    else:
        return Q0.style.set_properties(**{'background-color': 'black',
                                     'color': 'skyblue',
                                      'font-size':'small',
                                     'border-color': 'white'})\
            .render()
    

def getAED(name,conc_ivt):
    R = []
    for days_ivv,days_ivt,dm in [[i,j,k] for i in [90,365*3] for j in [1,2,3] for k in ['cave','cmax','auc']]:
        dose_ivv = calcAED(name,species='Rat',days_ivt=days_ivt,
                    days_ivv=days_ivv,
                    dose_ivt=conc_ivt,dose_metric=dm,
                    dbc_iviv=HCI.heprn_iviv)
        R.append(dict(name=name,days_ivv=days_ivv, days_ivt=days_ivt,dose_metric=dm,AED=dose_ivv))
        
    #X = pd.DataFrame(R).set_index(['name','days_ivt','days_ivv','dose_metric'])
    X = pd.DataFrame(R)[['name','days_ivt','days_ivv','dose_metric','AED']]
#     St = [dict(selector='row_heading',props=[("font-size","80%")]),
#           dict(selector='col_heading',props=[("font-size","80%")]),
#            dict(selector='td',props=[("font-size","80%")])]    
#     return X.style.set_table_styles(St).render()
    return X.style.set_properties(**{'background-color': 'black',
                                    'color': 'lawngreen',
                                     'font-size':'small',
                                    'border-color': 'white'})\
            .render()

    

def getHitInfo(HI):
    H = HCI.heprn_hitqual.find_one(dict(name=HI.name,ft_ch=HI.ft_ch,timeh=HI.timeh))
    return Box(H) if H else None

def saveHitInfo(HI):
    sav=Qual_save.value
    if HI.quality_flags=='other': HI.quality_flags=HI.quality_other
    if sav=='Append':
        if '_id' in HI: HI.pop('_id')
        #Log_info.value = Log_info.value + "<H2> Saving ..</H2>\n" + '<br>'.join(HI.keys())
    elif sav=='Del_prev':
        if '_id' in HI: HCI.heprn_hitqual.delete_one(HI)
        HCI.heprn_hitqual.delete_many(dict(name=HI.name,ft_ch=HI.ft_ch,timeh=HI.timeh,user=HI.user))
    HI.ft = HI.ft_ch[:2]
    HCI.heprn_hitqual.insert_one(HI.to_dict())    
    
def updateHitInfo(HI_new,user='ishah'):
    HI_old = getHitInfo(HI_new)
    HI = HI_old if HI_old else HI_new
    Hit_status.value= 'Reviewed' if ('_id' in HI and 'user' in HI) else 'Not Reviewed'
    Ch_name.value=HI.name 
    Asy_name.value=HI.ft_ch
    Ch_timeh.value=str(HI.timeh)
    Ch_ac50.value=str(HI.ac50)
    if 'quality_flags' in HI: Opt_qual.value = HI.quality_flags
    if 'quality_other' in HI: Qual_other.value=HI.quality_other
    Qual_user.value= HI.user if 'user' in HI else user
    Opt_hit.value='Yes' if 'hit' in HI and HI.hit else 'No'

    
def updateQualFlags():
    Flags = HCI.heprn_hitqual.distinct('quality_flags') + ['other']
    Opt_qual.options=Flags

def updateAED(HI):
    AED = None
    try:
        AED = getAED(HI.name,HI.ac50)
    except:
        AED_info.value="<b> No AED </b>"
    else:
        AED_info.value=AED
        
def updateHits(HI):
    H = getHitSummary(HI.name)
    Hit_info.value=H
# Callbacks


def on_change_rev(b):
    #Log_info.value = Log_info.value + "Switching to reviewed: %s <br>\n" % O1_rev.value
    Hits_new = getHits(reviewed=O1_rev.value,chemical=T1_chem.value,HITS=AC50)
    S1_hits.options=Hits_new.hit

def on_change_chm(b):
    if b.new == b.old: return
    Hits_new = getHits(reviewed=O1_rev.value,chemical=T1_chem.value,HITS=AC50)
    S1_hits.options=Hits_new.hit

def on_press_plot(b):
    h = S1_hits.value
    r1= R1.value
    chem,th,ft_ch,ac50 = re.match('^\s?([^\|]+)\|(\d+)h\|(\w+)\|\s?(\S+)\s?uM',h).groups()
    chem = chem.strip()
    th = int(th)
    ac50= float(ac50)
    H1.value=r"<b>Values %s</b>" % (S1_hits.value)
    
    #Log_info.value = Log_info.value + "<b> Loading %s </b><br>\n" % h
    HQ.update(dict(name=chem,ft_ch=ft_ch,timeh=th,ac50=ac50))
    # Load existing hits

    updateHitInfo(HQ)    
    updateAED(HQ)
    updateQualFlags()
    updateHits(HQ)
    
    pl.close('all')
    with Viz1:
        fig1 = plotFits(chem,nc=1,timeh=th,assay_comp=ft_ch,
                       Models=['hill','gnls'],r_max=r1,
                       dbc=HCI.heprn_cr,show_bmds=dict(E=[50],Z=[1]))
        pl.show(fig1)
        clear_output(wait=True)
    
    with Viz2:
        fig2 = plotAllFits(chem,dbc_cr=HCI.heprn_cr,dbc_qual=HCI.heprn_hitqual,
                           show_bmds=dict(E=[50]),
                           timeh=[1,3,6,24,48,72],
                           cols=dict(data='blue',fit='green',ac50='red',bmad='skyblue'),
                           r_max=r1,dbg=False,xs=2,ys=1.6,show_legend=True,
                           title_fmt="%(ft)s %(timeh)ih",fnsz=7)    
#         fig2 = plotFits(chem,nc=6,timeh=[6,24,48,72],assay_comp=FT_CH,
#                        Models=['hill','gnls'],r_max=r1,
#                        dbc=HCI.heprn_cr,show_bmds=dict(E=[50],Z=[1]))
        pl.show(fig2)
        clear_output(wait=True)
        
def on_press_save(b):
    #Log_info.value = Log_info.value + "<h3> Saving CR info </h3>\n" 
    HQ.hit = 1 if Opt_hit.value=='Yes' else 0
    HQ.quality_flags= Opt_qual.value
    HQ.quality_other= Qual_other.value
    HQ.user = Qual_user.value
    
    #Log_info.value = Log_info.value + '<br>\t\n'.join([k+':'+str(v) for k,v in HQ.iteritems()])
    
    saveHitInfo(HQ)
    updateHits(HQ)
    
## Calbacks
B1.on_click(on_press_plot)
B2.on_click(on_press_save)
O1_rev.observe(on_change_rev,names='value')
T1_chem.observe(on_change_chm,names='value')
display(L3)



# Add Ft to hitquality

In [81]:
pd.DataFrame(list(HCI.heprn_hitqual.find(dict(name='Triticonazole'))))

,_id,ac50,ft,ft_ch,hit,name,quality_flags,quality_other,timeh,user
0,5ceeca16b9e11e6b2591bc61,1.01,NS,NS_dn,0,Triticonazole,lt_3bmad,,24,ishah
1,5ceed115b9e11e1e5846385f,5.10,MF,MF_dn,0,Triticonazole,one-above-3bmad,,24,ishah
2,5ceed212b9e11e1e5846386a,7.05,ES,ES_up,0,Triticonazole,one-above-3bmad,,72,ishah
3,5ceed5aab9e11e1e58463888,12.41,St,St_dn,0,Triticonazole,one-above-3bmad,,72,ishah
4,5ceeda72b9e11e1e584638a2,20.25,ES,ES_up,0,Triticonazole,one-above-3bmad,,24,ishah
5,5cf00c14b9e11e1629e8ea70,4.05,DT,DT_dn,0,Triticonazole,lt_3bmad,,72,ishah
6,5cf00cd6b9e11e1629e8ea71,24.56,Ap,Ap_up,0,Triticonazole,one-above-3bmad,,72,ishah
7,5cf0410bb9e11e3c4ec7e1ca,57.94,St,St_up,0,Triticonazole,one-above-3bmad,,24,ishah


In [79]:
# for X in HCI.heprn_hitqual.find():
#     ft=X['ft_ch'][:2]
#     HCI.heprn_hitqual.update_one(X,{'$set':dict(ft=ft)})

In [15]:
# #HCI.heprn_hitqual.update_many(dict(quality_flags='bmad_lt_3'),{'$set':dict(quality_flags='lt_3bmad')})
# #AC50.columns

# #Opt_hit.value,Opt_qual.value,Qual_other.value,Qual_user.value
# #HQ.to_dict()
# X = pd.DataFrame(list(HCI.heprn_hitqual.find({},dict(_id=0))))
# X[X.name.str.match('2,4-Dichlorophenoxyacetic acid')]


In [16]:
HCI.heprn_hitqual.delete_one(dict(name='Chlorpropham',ft_ch='DT_dn',timeh=72,hit=0))

In [18]:
pd.DataFrame(list(HCI.heprn_hitqual.find(dict(name='Chlorpropham'),dict(_id=0))))

,ac50,ft_ch,hit,name,quality_flags,quality_other,timeh,user
0,0.22,NS_dn,0,Chlorpropham,lt_3bmad,,48,ishah
1,0.29,LM_dn,1,Chlorpropham,lt_3bmad,,72,ishah
2,0.40,ES_dn,0,Chlorpropham,lt_3bmad,,24,ishah
3,0.42,St_dn,0,Chlorpropham,lt_3bmad,,48,ishah
4,0.49,MF_dn,0,Chlorpropham,unclear_hill_fit,unclear_hill_fit,24,ishah
5,0.50,LM_dn,0,Chlorpropham,lt_3bmad,,24,ishah
6,2.23,ES_dn,0,Chlorpropham,one-above-3bmad,,72,ishah
7,83.04,NS_dn,0,Chlorpropham,lt_3bmad,,24,ishah
8,0.29,LM_dn,0,Chlorpropham,lt_3bmad,,72,ishah
9,0.29,LM_dn,0,Chlorpropham,lt_3bmad,,72,ishah


In [60]:
HCI.heprn_hitqual.find(dict(hit=1)).distinct('quality_flags')

[u'noisy',
 u'borderline-active',
 u'lt_3bmad',
 u'ok',
 u'monotonic-increase',
 u'one-above-3bmad',
 u'lt-3bmad-cr',
 u'one-above-3bmad-cr',
 u'monotnic-inc-baseline-up',
 u'unclear-gnls-fit']

In [88]:
HTML(getAED('Isazofos',100))

,name,days_ivt,days_ivv,dose_metric,AED
0,Isazofos,1,90,cave,2.1e+03
1,Isazofos,1,90,cmax,3e+02
2,Isazofos,1,90,auc,2.1e+03
3,Isazofos,2,90,cave,2.1e+03
4,Isazofos,2,90,cmax,3e+02
5,Isazofos,2,90,auc,2.1e+03
6,Isazofos,3,90,cave,2.1e+03
7,Isazofos,3,90,cmax,3e+02
8,Isazofos,3,90,auc,2.1e+03
9,Isazofos,1,1095,cave,2.1e+03


In [138]:
# for X in HCI.heprn_hitqual.find(dict(quality_flags='other')):
#     HCI.heprn_hitqual.update_one(X,{'$set':dict(quality_flags=X['quality_other'])})
    

In [28]:
h = S1_hits.value
h,re.match('^\s?([^\|]+)\|(\d+)h\|(\w+)\|(.*)uM$',h).groups()


(u'Hexaconazole|24h|ES_dn| 0.21uM',
 (u'Hexaconazole', u'24', u'ES_dn', u' 0.21'))

In [ ]:
Q = dict(name='2,4-Dichlorophenoxyacetic acid',timeh=24,ft='DT')
Q['best_fit.model']='hill'
#F0 = pd.DataFrame([f['best_fit'] for f in HCI.heprn_cr.find(Q,dict(_id=0))])
#F0.T

HCI.heprn_cr.find_one(Q,dict(_id=0))

In [43]:
AC50[AC50.name=='Dimethenamid'].sort_values('ac50').head().T

,143,136,134,140,139
aic,53,30,27,36,60
bmad,1,-1,1,-1,-1
ch,1,-1,1,-1,-1
data,z_ctrl,z_ctrl,z_ctrl,z_ctrl,z_ctrl
er,0.64,-0.56,-0.67,-0.18,1
er_sd,0.28,0.31,0.29,0.27,0.28
ft,ES,NS,DT,NS,MF
ft_ch,ES_up,NS_dn,DT_up,NS_dn,MF_dn
ft_type,single,single,single,single,single
ac50,71,76,79,80,82


In [11]:
# import seaborn as sns

# cm = sns.light_palette("orange", as_cmap=True)

# #X.style.bar(color='green')
# X.style.background_gradient(cmap=cm)

In [12]:
# # 1
# # html = X.style.set_table_styles(S1)
# # html
# #.set_properties(**{'font-size':'5pt'})\
# html = (X.style\
#          .set_table_styles(S1)\       
#          .render()
#        )
# html